# 1. 준비
## 1.1. 라이브러리 로드

In [ ]:
import os
from typing import Tuple, List, Sequence, Callable

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch

from torch import nn, Tensor
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision.models import mobilenet_v2
from torchvision.models import resnet101

# from torchvision.models.detection import keypointrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection import KeypointRCNN
from torch.utils.data.sampler import SubsetRandomSampler

!pip install -U git+https://github.com/albu/albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-j0anvkdm
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-j0anvkdm
     |████████████████████████████████| 952kB 6.0MB/s 
  Created wheel for albumentations: filename=albumentations-0.5.2-cp37-none-any.whl size=88322 sha256=547a5aa8ebf6ec21f83d5b7b4c83382ae6289d53fa556837235fc434aafffca2
  Stored in directory: /tmp/pip-ephem-wheel-cache-jbjfyto7/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


## 1.2 Google Drives 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

import os
os.chdir('/content/gdrive/Shared drives/keypoint/') # MyDrive/motion_keypoint/... (공유 풀더 위치)

Mounted at /content/gdrive/


In [ ]:
# draw_keypoints
# 기존 함수에서 boxpoints 추가함.
# boxpoints = [] 로 비워두면 자동으로 keypoints 값 이용해서 min, max 값 생성..
# test data 의 경우 채워두면 predicted box points 사용 

edges = [
    (0, 1), (0, 2), (2, 4), (1, 3), (6, 8), (8, 10), (9, 18),
    (10, 19), (5, 7), (7, 9), (11, 13), (13, 15), (12, 14),
    (14, 16), (15, 22), (16, 23), (20, 21), (5, 6), (5, 11),
    (6, 12), (11, 12), (17, 20), (20, 21), 
]

df2 = pd.read_csv('./data/mpii_cooc_label.csv')
df = pd.read_csv('./data/train_df2.csv')
keypoint_names = df.columns[1:].tolist()

def draw_keypoints(image: np.ndarray, keypoints: np.ndarray, boxpoints, edges: List[Tuple[int, int]]) -> None:
  """
  Args:
  image (ndarray): [H, W, C]
  keypoints (ndarray): [N, 3]
  edges (List(Tuple(int, int))):
  """
  np.random.seed(42)
  colors = {k: tuple(map(int, np.random.randint(0, 255, 3))) for k in range(24)}
  
  keypoints=keypoints.astype(np.int64)

  if len(boxpoints) == 0 :
    x1, y1 = min(keypoints[:, 0]), min(keypoints[:, 1])
    x2, y2 = max(keypoints[:, 0]), max(keypoints[:, 1]) 
  else :
    x1 = boxpoints[0]; y1 = boxpoints[1]; 
    x2 = boxpoints[2]; y2 = boxpoints[3];

  print(x1, y1, x2, y2)
  cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 100, 91), thickness=3)

  for i, keypoint in enumerate(keypoints):
    cv2.circle(image, tuple(keypoint),
               3, colors.get(i), thickness=3, lineType=cv2.FILLED)
    
    cv2.putText(image, f'{i}: {keypoint_names[i]}',
                tuple(keypoint),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    for i, edge in enumerate(edges):
      cv2.line(image, tuple(keypoints[edge[0]]),
               tuple(keypoints[edge[1]]), colors.get(edge[0]), 3, lineType=cv2.LINE_AA)
    
  fig, ax = plt.subplots(dpi=200)
  ax.imshow(image)
  ax.axis('off')
  plt.show()
  # fig.savefig('example.png')

In [ ]:
df2.head()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y,left_shoulde,right_shoulder,left_elbow,right_elbow,left_wrist,right_wrist,left_hip,right_hip,left_knee,right_knee,left_ankle,right_ankle,neck,left_palm,right_palm,spine2(back),spine1(waist),left_instep,right_instep
0,036636184.jpg,419.813812,74.909950,418.614716,59.914700,414.417786,68.911850,445.594818,38.321537,446.793915,37.721729,516,48,459,104,588,132,469,201,587,242,416,251,570,204,517,220,574,371,529,369,581,480,525,478,479.0873,71.5436,NaN,NaN,NaN,NaN,488,76,544,212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,045606998.jpg,409.908722,177.177002,418.299591,169.383072,462.651398,221.542343,492.618835,212.549347,490.820740,219.743729,566,187,464,170,671,230,470,285,591,316,495,361,645,140,559,255,600,343,568,298,610,480,555,370,508.3448,181.3768,NaN,NaN,NaN,NaN,515,179,602,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,060754485.jpg,607.660767,171.195572,618.196838,157.751541,593.293335,161.592697,645.973938,158.711823,584.672913,172.155853,698,217,600,242,767,310,551,307,790,405,488,351,747,421,670,437,751,574,685,579,768,717,694,684,642.6337,217.5659,NaN,NaN,NaN,NaN,649,230,709,429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,017052412.jpg,571.060974,167.434402,584.473206,154.949890,557.648743,157.830948,608.423645,154.949890,546.152527,165.513702,636,220,553,217,616,329,526,310,529,353,495,354,748,369,639,374,727,532,663,528,727,666,679,637,592.5583,210.0951,NaN,NaN,NaN,NaN,595,219,694,372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,094888554.jpg,877.375122,356.501862,886.957764,352.665131,873.541992,348.828369,895.582153,361.297821,862.042786,352.665131,907,404,844,390,898,460,817,441,862,504,775,457,900,515,830,500,895,593,841,595,912,691,836,666,875.7539,390.6023,NaN,NaN,NaN,NaN,876,397,865,508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(df2.shape)

(2093, 68)


In [ ]:
mpii_info = pd.read_csv('./data/mpii_cooc_label.csv')

In [ ]:
motion_key = ["nose_x", "nose_y",
              "left_eye_x", "left_eye_y",
              "right_eye_x", "right_eye_y",
              "left_ear_x", "left_ear_y",
              "right_ear_x", "right_ear_y",
              "left_shoulder_x", "left_shoulder_y",

               "right_shoulder_x", "right_shoulder_y",
               "left_elbow_x", "left_elbow_y",
               "right_elbow_x", "right_elbow_y",
               "left_wrist_x", "left_wrist_y",
               "right_wrist_x", "right_wrist_y",
               "left_hip_x", "left_hip_y",
               "right_hip_x", "right_hip_y",
               "left_knee_x", "left_knee_y",
               "right_knee_x", "right_knee_y",
               "left_ankle_x", "left_ankle_y",
               "right_ankle_x", "right_ankle_y",
               "neck_x", "neck_y",
               "left_palm_x", "left_palm_y",
               "right_palm_x", "right_palm_y",
               "spine2(back)_x", "spine2(back)_y",
               "spine1(waist)_x", "spine1(waist)_y",
               "left_instep_x", "left_instep_y",
               "right_instep_x", "right_instep_y"]

print(len(motion_key))
             
# left_sholuder
motion_vis = ["left_shoulde", "right_shoulder", "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", "right_knee", "left_ankle", "right_ankle", "neck", "left_palm", "right_palm", "spine2(back)", "spine1(waist)", "left_instep", "right_instep"]

mpii_info['left_palm'] = mpii_info['left_wrist']
mpii_info['left_palm_x'] = mpii_info['left_wrist_x']
mpii_info['left_palm_y'] = mpii_info['left_wrist_y']

mpii_info['right_palm'] = mpii_info['right_wrist']
mpii_info['right_palm_x'] = mpii_info['right_wrist_x']
mpii_info['right_palm_y'] = mpii_info['right_wrist_y']

mpii_info['left_instep'] = mpii_info['left_ankle']
mpii_info['left_instep_x'] = mpii_info['left_ankle_x']
mpii_info['left_instep_y'] = mpii_info['left_ankle_y']

mpii_info['right_instep'] = mpii_info['right_ankle']
mpii_info['right_instep_x'] = mpii_info['right_ankle_x']
mpii_info['right_instep_y'] = mpii_info['right_ankle_y']

print(mpii_info.shape)
print(mpii_info[motion_key].shape)
print(mpii_info[motion_vis].shape)


48
(2093, 68)
(2093, 48)
(2093, 19)


In [ ]:
mpii_info.head()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y,left_shoulde,right_shoulder,left_elbow,right_elbow,left_wrist,right_wrist,left_hip,right_hip,left_knee,right_knee,left_ankle,right_ankle,neck,left_palm,right_palm,spine2(back),spine1(waist),left_instep,right_instep
0,036636184.jpg,419.813812,74.909950,418.614716,59.914700,414.417786,68.911850,445.594818,38.321537,446.793915,37.721729,516,48,459,104,588,132,469,201,587,242,416,251,570,204,517,220,574,371,529,369,581,480,525,478,479.0873,71.5436,587,242,416,251,488,76,544,212,581,480,525,478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,045606998.jpg,409.908722,177.177002,418.299591,169.383072,462.651398,221.542343,492.618835,212.549347,490.820740,219.743729,566,187,464,170,671,230,470,285,591,316,495,361,645,140,559,255,600,343,568,298,610,480,555,370,508.3448,181.3768,591,316,495,361,515,179,602,198,610,480,555,370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,060754485.jpg,607.660767,171.195572,618.196838,157.751541,593.293335,161.592697,645.973938,158.711823,584.672913,172.155853,698,217,600,242,767,310,551,307,790,405,488,351,747,421,670,437,751,574,685,579,768,717,694,684,642.6337,217.5659,790,405,488,351,649,230,709,429,768,717,694,684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,017052412.jpg,571.060974,167.434402,584.473206,154.949890,557.648743,157.830948,608.423645,154.949890,546.152527,165.513702,636,220,553,217,616,329,526,310,529,353,495,354,748,369,639,374,727,532,663,528,727,666,679,637,592.5583,210.0951,529,353,495,354,595,219,694,372,727,666,679,637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,094888554.jpg,877.375122,356.501862,886.957764,352.665131,873.541992,348.828369,895.582153,361.297821,862.042786,352.665131,907,404,844,390,898,460,817,441,862,504,775,457,900,515,830,500,895,593,841,595,912,691,836,666,875.7539,390.6023,862,504,775,457,876,397,865,508,912,691,836,666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# cut off 참고 : # https://github.com/uoguelph-mlrg/Cutout/blob/master/util/cutout.py
# p=0.5 의 확률로 cutout image를 생성함 / 패치 크기는 150x150 으로 고정
class MotionDataset(Dataset):
  def __init__(self, label_df, transforms=None) :
      self.df = label_df
      self.transforms = transforms
  
  def __len__(self) -> int:
    return self.df.shape[0]
    
  def __getitem__(self,index: int) -> Tuple[Tensor]:
#    print(index)
    image_dir = self.df.iloc[index,]['path']
    image_id = self.df.iloc[index,]['image']
    data_type = self.df.iloc[index,]['type']
    
    labels = np.array([1]) 
    keypoints = self.df.iloc[index, ][motion_key].values.reshape(-1, 2).astype(np.int64)

    x1, y1 = min(keypoints[:, 0]), min(keypoints[:, 1])
    x2, y2 = max(keypoints[:, 0]), max(keypoints[:, 1])
    boxes = np.array([[x1, y1, x2, y2]], dtype=np.int64)
    
    image = cv2.imread(os.path.join(image_dir, image_id), cv2.COLOR_BGR2RGB)
    #if data_type == "mpii" : 
    #  visible = self.df.iloc[index, ][motion_vis].values.reshape(19,1).astype(np.int64)
    #else : 
    #  visible = np.ones((19, 1))
    visible = np.ones((24, 1))
    
    # keypoints-aware occlusion p=0.5
    cutout = np.random.randint(0,2)
    if cutout == 1:
      h = image.shape[0]
      w = image.shape[1]

      #item = np.random.randint(0,19)
      item = np.random.randint(0,24)
      initx, inity = keypoints[item]
      size = np.random.randint(200, 300)

      image = cv2.rectangle(image, (initx-size, inity-size), (initx+size, inity+size), (0,0,0), -1)

      for i, keypoint in enumerate(keypoints):
        invis_check = sum([initx, inity-size] < keypoint) + sum(keypoint < [initx+size, inity])
        if invis_check == 4:
          visible[i]=0

    targets ={
        'image': image,
        'bboxes': boxes,
        'labels': labels,
        'keypoints': keypoints
    }
    
    if self.transforms is not None:
      targets = self.transforms(**targets)
      
    image = targets['image']
    image = image / 255.0

    targets = {
        'labels': torch.as_tensor(targets['labels'], dtype=torch.int64),
        'boxes': torch.as_tensor(targets['bboxes'], dtype=torch.float32),
        'keypoints': torch.as_tensor(
                np.concatenate([targets['keypoints'], visible], axis=1)[np.newaxis], dtype=torch.float32 )
        }

    return image, targets

In [ ]:
def check_bbox(bbox):
    """Check if bbox boundaries are in range 0, 1 and minimums are lesser then maximums"""
    for name, value in zip(["x_min", "y_min", "x_max", "y_max"], bbox[:4]):
        if not 0 <= value <= 1:
          bbox=list(bbox)
          for i in range(4):
            if (bbox[i]<0) :
              bbox[i]=0.00001
            elif (bbox[i]>1) :
              bbox[i]=0.99999
          bbox=tuple(bbox)
   #end of block
    x_min, y_min, x_max, y_max = bbox[:4]
    if x_max <= x_min:
        raise ValueError("x_max is less than or equal to x_min for bbox {bbox}.".format(bbox=bbox))
    if y_max <= y_min:
        raise ValueError("y_max is less than or equal to y_min for bbox {bbox}.".format(bbox=bbox))

A.bbox_utils.check_bbox = check_bbox

In [ ]:
# https://github.com/aleju/imgaug
# https://imgaug.readthedocs.io/en/latest/source/overview/blur.html#gaussianblur
transform1 = A.Compose([A.Resize(800, 1333), 
                       A.Rotate(limit=45, p=1.0),
                       A.OneOf([A.MedianBlur(p=1.0),
                                A.ImageCompression(p=1.0),
                       ],p=0.5),
#                       A.VerticalFlip(p=0.5),
                       A.RandomScale(scale_limit=0.25, p=1.0),
                       ToTensorV2(),
                       ],
                      bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']),
                      keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

transform_val = A.Compose([ToTensorV2(),
                       ],
                      bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']),
                      keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

In [ ]:
def collate_fn(batch: torch.Tensor) -> Tuple:
    return tuple(zip(*batch))

train_path = './data/train_imgs/'
train_df = pd.read_csv('./data/relabel_all_df.csv')
train_df = pd.read_csv('./data/train_df2.csv')
train_df['path'] = train_path
train_df['type'] = "dacon"

mpii_path = './data/external_labeled/exter_imgs/images/'
mpii_info['path'] = mpii_path
mpii_info['type'] = "mpii"

train_all = pd.concat([train_df, mpii_info], ignore_index=True) 

In [ ]:
# R-CNN
def get_model() -> nn.Module:
    mode = resnet101(pretrained=True)
    backbone = nn.Sequential(*(list(nn.Module.children(mode))[0:8]))
    backbone.out_channels = 2048
    
    roi_pooler = MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))
    
    keypoint_roi_pooler = MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=14,
        sampling_ratio=2
    )

    model = KeypointRCNN(
        backbone, 
        num_classes=2,
        num_keypoints=24,
        box_roi_pool=roi_pooler,
        keypoint_roi_pool=keypoint_roi_pooler,
        rpn_anchor_generator=anchor_generator
    )

    return model

In [ ]:
import random
from tqdm import tqdm
from sklearn.model_selection import KFold
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# FOLD 5

In [ ]:
# 5-CV 추가
# 참고 : https://dacon.io/competitions/official/235697/codeshare/2446?page=1&dtype=recent
num_epochs = 30
SEED = 42
lam=2**(-20)
device='cuda:0'

# cross validation을 적용하기 위해 KFold 생성
kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)

# dirty_mnist_answer에서 train_idx와 val_idx를 생성
#best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장

for fold_index, (trn_idx, val_idx) in enumerate(kfold.split( train_all ),1):
    if fold_index == 5 :
      break

print(trn_idx)
print(val_idx)

[   0    1    2 ... 6184 6185 6186]
[   3    4    5 ... 6173 6175 6178]


In [ ]:
EARLY_STOPPING_EPOCH = 4

In [ ]:
# for fold_index, (trn_idx, val_idx) in enumerate(kfold.split( train_all ),1):

seed_everything(SEED)
print(f' == fold: {fold_index} == ')

# cuda cache 초기화
torch.cuda.empty_cache()

#train fold, validation fold 분할
trainset = train_all.iloc[trn_idx]
validset  = train_all.iloc[val_idx]

#Dataset 정의
train_data = MotionDataset(label_df=trainset, transforms=transform1)
valid_data = MotionDataset(label_df=validset, transforms=transform_val)

#DataLoader 정의
train_loader = DataLoader(train_data, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)
valid_loader = DataLoader(valid_data, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)

# 모델 선언
model = get_model()
model.to(device)# gpu에 모델 할당

optimizer = optim.Adam(model.parameters(), lr=1e-4)
lr_sched = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

valid_early_stop = 0
valid_best_loss = float('inf')

for epoch in range(num_epochs):  
  model.train()
  for i, (images, targets) in enumerate(train_loader):
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    optimizer.zero_grad()
    model.train()
    losses = model(images, targets)
    loss = sum(loss for loss in losses.values())
    
    L2=0
    for param in model.parameters():
      L2 += torch.norm(param,p=2)
      loss = loss + lam * L2

    loss.backward()
    optimizer.step()

    if (i+1) % 20 == 0:
      print(f'| epoch: {epoch} | Train loss: {loss.item():.4f}', end=' | ')
      for k, v in losses.items():
        print(f'{k[5:]}: {v.item():.4f}', end=' | ')
      print()
  
  # Learning rate 조절
  lr_sched.step()

  if epoch < 1 : 
    continue

  valid_loss_list = []
  for i, (images, targets) in enumerate(valid_loader):
    optimizer.zero_grad()
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
      
    with torch.no_grad():
      losses  = model(images, targets)
      loss = sum(loss for loss in losses.values())

      valid_loss_list.append(losses['loss_keypoint'])

    if (i+1) % 20 == 0:
      print(f'| epoch: {epoch} | Valid loss: {loss.item():.4f}', end=' | ')
      for k, v in losses.items():
        print(f'{k[5:]}: {v.item():.4f}', end=' | ')
      print()

  val_mean_loss = np.mean(valid_loss_list, dtype="float64")
  print(f' > VALIDATION SET LOSS MEAN : {val_mean_loss : .4f}')
  
  # 모델 저장
  # 성능 변화 없는 에폭 3번 반복 -> Early stopping
  if val_mean_loss  < valid_best_loss :
    best_model = model
    MODEL = f"resnet101_{SEED}"
    # 경로 설정 필요
    path = f"./model/5CV/SEED_{SEED}/"
    try: 
      os.mkdir(path)
    except:
      None 
    torch.save(best_model.state_dict(), f'{path}fold{fold_index}]_{MODEL}_{val_mean_loss:2.4f}_epoch_{epoch}.pth')

  # early stopping    
  if val_mean_loss < valid_best_loss:
    valid_best_loss = val_mean_loss
    valid_early_stop = 0
  else:
    valid_early_stop += 1
    if valid_early_stop >= EARLY_STOPPING_EPOCH:
      print("EARLY_STOPPING!!")
      break


# 폴드별로 가장 좋은 모델 저장
# best_models.append(best_model)

 == fold: 5 == 


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth



| epoch: 0 | Train loss: 7.7522 | classifier: 0.0833 | box_reg: 0.0342 | keypoint: 7.3721 | objectness: 0.0556 | rpn_box_reg: 0.0183 | 
| epoch: 0 | Train loss: 7.6933 | classifier: 0.1397 | box_reg: 0.1008 | keypoint: 7.2019 | objectness: 0.0378 | rpn_box_reg: 0.0244 | 
| epoch: 0 | Train loss: 8.4481 | classifier: 0.1027 | box_reg: 0.1092 | keypoint: 7.9140 | objectness: 0.0719 | rpn_box_reg: 0.0615 | 
| epoch: 0 | Train loss: 7.2843 | classifier: 0.0920 | box_reg: 0.0723 | keypoint: 6.8478 | objectness: 0.0487 | rpn_box_reg: 0.0346 | 
| epoch: 0 | Train loss: 7.6419 | classifier: 0.0937 | box_reg: 0.1408 | keypoint: 7.1843 | objectness: 0.0271 | rpn_box_reg: 0.0069 | 
| epoch: 0 | Train loss: 7.8492 | classifier: 0.1049 | box_reg: 0.1090 | keypoint: 7.3875 | objectness: 0.0425 | rpn_box_reg: 0.0162 | 
| epoch: 0 | Train loss: 7.3157 | classifier: 0.1200 | box_reg: 0.1146 | keypoint: 6.8115 | objectness: 0.0523 | rpn_box_reg: 0.0282 | 
| epoch: 0 | Train loss: 8.3739 | classifier: 0

## 예측

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.head()
submission.shape

In [ ]:
# submission file 
images_name = []
predicted = []
device = "cuda:0"

best_model.eval()
best_model.to(device)

path = './data/test_imgs/'
i=0
with torch.no_grad():
  for img_name in submission['image']:
    image = cv2.imread(path + img_name, cv2.COLOR_BGR2RGB)

    image = image / 255.0
    image = image.transpose(2, 0, 1)
    image = [torch.as_tensor(image, dtype=torch.float32, device=device)]

    Motionpreds = best_model(image)
    
    try : 
      Motionkeypoints = Motionpreds[0]['keypoints'].cpu().detach().numpy().copy()[0]
      
    except :
      print(img_name)
      print("predicted value is empty. \n")

    images_name.append(img_name)
    Motionkeypoints2 = Motionkeypoints[:, :2].copy()

    image = cv2.imread(path + img_name, cv2.COLOR_BGR2RGB)
    if i % 100 == 0 :
      print(i)
      draw_keypoints(image, Motionkeypoints2, [], edges)
    i=i+1

    motion_keypoints = [item for sublist in Motionkeypoints2 for item in sublist]
    #print(flat_keypoints)
    predicted.append(motion_keypoints)

In [ ]:
print(len(predicted)) 
print(len(images_name))

In [ ]:
sub = pd.DataFrame(columns=submission.columns)

all_predictions = np.array(predicted)
sub['image'] = images_name
sub.iloc[:,1:] = all_predictions

sub.to_csv('./code/5CV-hr/sub_5fold_0330_best.csv', index=False)